In [12]:
import os
import csv
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta

sc

In [49]:
def deltadates(start, end):
    while start <= end:
        yield '{0}{1:02d}{2:02d}'.format(start.year, start.month, start.day)
        start += timedelta(days=1)

def downloaddata(folder, downloadday):
    filename = 'bus_time_' + downloadday + '.csv.xz'
    year = str(downloadday[0:4])
    month = str(downloadday[4:6])
    urlloc = 'http://data.mytransit.nyc.s3.amazonaws.com/bus_time/' 
    url = '{0}{1}/{1}-{2}/{3}'.format(urlloc, year, month, filename)
    
    os.system('curl -o ' + folder + filename + ' ' + url)
    os.system('xz -d ' + folder + filename )

def deletedata(folder, downloadday):
    filename = 'bus_time_' + downloadday
    os.remove(folder + filename + '.csv')
    
def extractbusdata(Id, records):
    if Id == 0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if any(b in row[7] for b in buses):
            yield row

In [51]:
folder = 'Data/'
buses = ['Q48', 'Bx6', 'Bx13', 'Bx1']

for d in deltadates(date(2014, 8, 9), date(2014, 8, 12)):
    
    downloaddata(folder, d)
    
    bus_data = sc.textFile(folder + 'bus_time_' + d + '.csv')
    data = bus_data.mapPartitionsWithIndex(extractbusdata)
    
    with open(folder + 'output.csv', 'wb') as f:
        w = csv.writer(f)
        w.writerows(data.collect())
    
    deletedata(folder, d)